##### First python implementation of the stylized decision problem described in the MatterMost article.

In [16]:
import numpy as np
import copy

In [17]:
# Defining the set of allowed states in the SDP.
states = ["DHU", "DHC", "DLU", "DLC", "SHU", "SHC", "SLU", "SLC"] 

In [18]:
# Defining the probabilities for the transition function.
pS_Start = 0.9
pD_Start = 1.0 - pS_Start

pD_Delay = 0.9
pS_Delay = 1.0 - pD_Delay

pL_S_DH = 0.7
pL_S_DL = 0.9
pL_S_SH = 0.3
pL_S_SL = 0.7

pH_S_DH = 1.0 - pL_S_DH
pH_S_DL = 1.0 - pL_S_DL
pH_S_SH = 1.0 - pL_S_SH
pH_S_SL = 1.0 - pL_S_SL

pL_D_DH = pL_S_SH
pL_D_DL = pL_S_SL
pH_D_DH = 1.0 - pL_D_DH
pH_D_DL = 1.0 - pL_D_DL

pU_S_0 = 0.9
pU_D_0 = 0.7
pC_S_0 = 1.0 - pU_S_0
pC_D_0 = 1.0 - pU_D_0

pU_S = 0.9
pU_D = 0.3
pC_S = 1.0 - pU_S
pC_D = 1.0 - pU_D

In [19]:
# Checking that no probabilities are negative.
def mkSimpleProb(pairs: list[tuple[str, float]]) -> dict[str, float]:
    dist: dict[str, float] = {}
    for (st, pr) in pairs:
        if pr >= 0:
            dist[st] = pr
    return dist

In [20]:
# For a current time step, state and action, returns the probabilities of entering each state in the next time step.
def next(t: int, x: str, y: str) -> dict[str, float]:
    # "y" can be "Start", "Delay", or None for S-states.
    if t == 0:
        # CASE: t == 0
        if x == "DHU":
            if y == "Start":
                return mkSimpleProb([
                    ("DHU", pD_Start * pH_D_DH * pU_D_0),
                    ("DHC", pD_Start * pH_D_DH * pC_D_0),
                    ("DLU", pD_Start * pL_D_DH * pU_D_0),
                    ("DLC", pD_Start * pL_D_DH * pC_D_0),
                    ("SHU", pS_Start * pH_S_DH * pU_S_0),
                    ("SHC", pS_Start * pH_S_DH * pC_S_0),
                    ("SLU", pS_Start * pL_S_DH * pU_S_0),
                    ("SLC", pS_Start * pL_S_DH * pC_S_0),
                ])
            elif y == "Delay":
                return mkSimpleProb([
                    ("DHU", pD_Delay * pH_D_DH * pU_D_0),
                    ("DHC", pD_Delay * pH_D_DH * pC_D_0),
                    ("DLU", pD_Delay * pL_D_DH * pU_D_0),
                    ("DLC", pD_Delay * pL_D_DH * pC_D_0),
                    ("SHU", pS_Delay * pH_S_DH * pU_S_0),
                    ("SHC", pS_Delay * pH_S_DH * pC_S_0),
                    ("SLU", pS_Delay * pL_S_DH * pU_S_0),
                    ("SLC", pS_Delay * pL_S_DH * pC_S_0),
                ])
            else:
                raise ValueError("Invalid control for DHU at t=0.")
        
        elif x == "DHC":
            if y == "Start":
                return mkSimpleProb([
                    ("DHC", pD_Start * pH_D_DH),
                    ("DLC", pD_Start * pL_D_DH),
                    ("SHC", pS_Start * pH_S_DH),
                    ("SLC", pS_Start * pL_S_DH),
                ])
            elif y == "Delay":
                return mkSimpleProb([
                    ("DHC", pD_Delay * pH_D_DH),
                    ("DLC", pD_Delay * pL_D_DH),
                    ("SHC", pS_Delay * pH_S_DH),
                    ("SLC", pS_Delay * pL_S_DH),
                ])
            else:
                raise ValueError("Invalid control for DHC at t=0.")
        
        elif x == "DLU":
            if y == "Start":
                return mkSimpleProb([
                    ("DHU", pD_Start * pH_D_DL * pU_D_0),
                    ("DHC", pD_Start * pH_D_DL * pC_D_0),
                    ("DLU", pD_Start * pL_D_DL * pU_D_0),
                    ("DLC", pD_Start * pL_D_DL * pC_D_0),
                    ("SHU", pS_Start * pH_S_DL * pU_S_0),
                    ("SHC", pS_Start * pH_S_DL * pC_S_0),
                    ("SLU", pS_Start * pL_S_DL * pU_S_0),
                    ("SLC", pS_Start * pL_S_DL * pC_S_0),
                ])
            elif y == "Delay":
                return mkSimpleProb([
                    ("DHU", pD_Delay * pH_D_DL * pU_D_0),
                    ("DHC", pD_Delay * pH_D_DL * pC_D_0),
                    ("DLU", pD_Delay * pL_D_DL * pU_D_0),
                    ("DLC", pD_Delay * pL_D_DL * pC_D_0),
                    ("SHU", pS_Delay * pH_S_DL * pU_S_0),
                    ("SHC", pS_Delay * pH_S_DL * pC_S_0),
                    ("SLU", pS_Delay * pL_S_DL * pU_S_0),
                    ("SLC", pS_Delay * pL_S_DL * pC_S_0),
                ])
            else:
                raise ValueError("Invalid control for DLU at t=0.")
        
        elif x == "DLC":
            if y == "Start":
                return mkSimpleProb([
                    ("DHC", pD_Start * pH_D_DL),
                    ("DLC", pD_Start * pL_D_DL),
                    ("SHC", pS_Start * pH_S_DL),
                    ("SLC", pS_Start * pL_S_DL),
                ])
            elif y == "Delay":
                return mkSimpleProb([
                    ("DHC", pD_Delay * pH_D_DL),
                    ("DLC", pD_Delay * pL_D_DL),
                    ("SHC", pS_Delay * pH_S_DL),
                    ("SLC", pS_Delay * pL_S_DL),
                ])
            else:
                raise ValueError("Invalid control for DLC at t=0.")
        
        elif x == "SHU":
            # In Idris: Theory.next Z SHU () = mkSimpleProb [...]
            # The control is '()', i.e. a unit (we represent it as None in Python).
            if y is None:
                return mkSimpleProb([
                    ("SHU", pH_S_SH * pU_S_0),
                    ("SHC", pH_S_SH * pC_S_0),
                    ("SLU", pL_S_SH * pU_S_0),
                    ("SLC", pL_S_SH * pC_S_0),
                ])
            else:
                raise ValueError("Invalid control for SHU at t=0 (should be None).")
        
        elif x == "SHC":
            if y is None:
                return mkSimpleProb([
                    ("SHC", pH_S_SH),
                    ("SLC", pL_S_SH),
                ])
            else:
                raise ValueError("Invalid control for SHC at t=0.")
        
        elif x == "SLU":
            if y is None:
                return mkSimpleProb([
                    ("SHU", pH_S_SL * pU_S_0),
                    ("SHC", pH_S_SL * pC_S_0),
                    ("SLU", pL_S_SL * pU_S_0),
                    ("SLC", pL_S_SL * pC_S_0),
                ])
            else:
                raise ValueError("Invalid control for SLU at t=0.")
        
        elif x == "SLC":
            if y is None:
                return mkSimpleProb([
                    ("SHC", pH_S_SL),
                    ("SLC", pL_S_SL),
                ])
            else:
                raise ValueError("Invalid control for SLC at t=0.")
        
        else:
            raise ValueError("Unexpected state at t=0.")
    
    else:
        # CASE: t > 0
        # The code is perfectly analogous but uses pU_S, pC_S, pU_D, pC_D 
        # instead of pU_S_0, pC_S_0, pU_D_0, pC_D_0.
        
        if x == "DHU":
            if y == "Start":
                return mkSimpleProb([
                    ("DHU", pD_Start * pH_D_DH * pU_D),
                    ("DHC", pD_Start * pH_D_DH * pC_D),
                    ("DLU", pD_Start * pL_D_DH * pU_D),
                    ("DLC", pD_Start * pL_D_DH * pC_D),
                    ("SHU", pS_Start * pH_S_DH * pU_S),
                    ("SHC", pS_Start * pH_S_DH * pC_S),
                    ("SLU", pS_Start * pL_S_DH * pU_S),
                    ("SLC", pS_Start * pL_S_DH * pC_S),
                ])
            elif y == "Delay":
                return mkSimpleProb([
                    ("DHU", pD_Delay * pH_D_DH * pU_D),
                    ("DHC", pD_Delay * pH_D_DH * pC_D),
                    ("DLU", pD_Delay * pL_D_DH * pU_D),
                    ("DLC", pD_Delay * pL_D_DH * pC_D),
                    ("SHU", pS_Delay * pH_S_DH * pU_S),
                    ("SHC", pS_Delay * pH_S_DH * pC_S),
                    ("SLU", pS_Delay * pL_S_DH * pU_S),
                    ("SLC", pS_Delay * pL_S_DH * pC_S),
                ])
            else:
                raise ValueError("Invalid control for DHU at t>0.")
        
        elif x == "DHC":
            if y == "Start":
                return mkSimpleProb([
                    ("DHC", pD_Start * pH_D_DH),
                    ("DLC", pD_Start * pL_D_DH),
                    ("SHC", pS_Start * pH_S_DH),
                    ("SLC", pS_Start * pL_S_DH),
                ])
            elif y == "Delay":
                return mkSimpleProb([
                    ("DHC", pD_Delay * pH_D_DH),
                    ("DLC", pD_Delay * pL_D_DH),
                    ("SHC", pS_Delay * pH_S_DH),
                    ("SLC", pS_Delay * pL_S_DH),
                ])
            else:
                raise ValueError("Invalid control for DHC at t>0.")
        
        elif x == "DLU":
            if y == "Start":
                return mkSimpleProb([
                    ("DHU", pD_Start * pH_D_DL * pU_D),
                    ("DHC", pD_Start * pH_D_DL * pC_D),
                    ("DLU", pD_Start * pL_D_DL * pU_D),
                    ("DLC", pD_Start * pL_D_DL * pC_D),
                    ("SHU", pS_Start * pH_S_DL * pU_S),
                    ("SHC", pS_Start * pH_S_DL * pC_S),
                    ("SLU", pS_Start * pL_S_DL * pU_S),
                    ("SLC", pS_Start * pL_S_DL * pC_S),
                ])
            elif y == "Delay":
                return mkSimpleProb([
                    ("DHU", pD_Delay * pH_D_DL * pU_D),
                    ("DHC", pD_Delay * pH_D_DL * pC_D),
                    ("DLU", pD_Delay * pL_D_DL * pU_D),
                    ("DLC", pD_Delay * pL_D_DL * pC_D),
                    ("SHU", pS_Delay * pH_S_DL * pU_S),
                    ("SHC", pS_Delay * pH_S_DL * pC_S),
                    ("SLU", pS_Delay * pL_S_DL * pU_S),
                    ("SLC", pS_Delay * pL_S_DL * pC_S),
                ])
            else:
                raise ValueError("Invalid control for DLU at t>0.")
        
        elif x == "DLC":
            if y == "Start":
                return mkSimpleProb([
                    ("DHC", pD_Start * pH_D_DL),
                    ("DLC", pD_Start * pL_D_DL),
                    ("SHC", pS_Start * pH_S_DL),
                    ("SLC", pS_Start * pL_S_DL),
                ])
            elif y == "Delay":
                return mkSimpleProb([
                    ("DHC", pD_Delay * pH_D_DL),
                    ("DLC", pD_Delay * pL_D_DL),
                    ("SHC", pS_Delay * pH_S_DL),
                    ("SLC", pS_Delay * pL_S_DL),
                ])
            else:
                raise ValueError("Invalid control for DLC at t>0.")
        
        elif x == "SHU":
            if y is None:
                return mkSimpleProb([
                    ("SHU", pH_S_SH * pU_S),
                    ("SHC", pH_S_SH * pC_S),
                    ("SLU", pL_S_SH * pU_S),
                    ("SLC", pL_S_SH * pC_S),
                ])
            else:
                raise ValueError("Invalid control for SHU at t>0 (should be None).")
        
        elif x == "SHC":
            if y is None:
                return mkSimpleProb([
                    ("SHC", pH_S_SH),
                    ("SLC", pL_S_SH),
                ])
            else:
                raise ValueError("Invalid control for SHC at t>0.")
        
        elif x == "SLU":
            if y is None:
                return mkSimpleProb([
                    ("SHU", pH_S_SL * pU_S),
                    ("SHC", pH_S_SL * pC_S),
                    ("SLU", pL_S_SL * pU_S),
                    ("SLC", pL_S_SL * pC_S),
                ])
            else:
                raise ValueError("Invalid control for SLU at t>0.")
        
        elif x == "SLC":
            if y is None:
                return mkSimpleProb([
                    ("SHC", pH_S_SL),
                    ("SLC", pL_S_SL),
                ])
            else:
                raise ValueError("Invalid control for SLC at t>0.")
        
        else:
            raise ValueError("Unexpected state at t>0.")

In [21]:
# Testing the transition function.
test = next(0, "DHU", "Start")

for k, v in test.items():
    print(k, v)
print("\nSum of all probabilities: ", sum(test.values()))

DHU 0.04899999999999998
DHC 0.020999999999999998
DLU 0.020999999999999994
DLC 0.009
SHU 0.24300000000000008
SHC 0.027
SLU 0.5670000000000001
SLC 0.06299999999999999

Sum of all probabilities:  1.0


In [22]:
# Reward function.
def reward(t: int, x: str, y: str, next_x: str) -> int:
    # Value is added for transitioning into states which do not have low economic
    # output and at the same time are not comitted to severe future climate change.
    return 1 if next_x in ["DHU", "SHU"] else 0

# print(reward(0, "DHU", "Start", "DHU"))

In [23]:
# Computing the total expected value from a policy sequence when starting at time t in state x.
def val(t: int, ps: list[dict[str, str]], x: str) -> float:
    value = 0
    if len(ps) == 0:
        return value
    y = ps[0][x]
    m_next = next(t, x, y)
    for x_prim, pr in m_next.items():
        value += (reward(t, x, y, x_prim) + val(t+1, ps[1:], x_prim)) * pr
    return value

In [24]:
# Test of val function.
ps_test_start = [{"DHU": "Start"}, 
           {"DHU": "Start", "DHC": "Start",
            "DLU": "Start", "DLC": "Start",
            "SHU": None, "SHC": None,
            "SLU": None, "SLC": None}]

ps_test_delay = [{"DHU": "Delay"}, 
           {"DHU": "Delay", "DHC": "Delay",
            "DLU": "Delay", "DLC": "Delay",
            "SHU": None, "SHC": None,
            "SLU": None, "SLC": None}]

print(val(0, ps_test_start, "DHU"), val(0, ps_test_delay, "DHU"))

0.6130060000000002 0.6142859999999999


In [25]:
# Computes the best single policy to add to an existing policy sequence.
def best_ext(t: int, ps_tail: list[dict[str, str]]) -> dict[str, str]:
    policy = dict()

    for state in states:
        best_value = -np.inf
        best_action = None

        if state in ["DHU", "DHC", "DLU", "DLC"]:
            actions = ["Start", "Delay"]
        else:
            policy[state] = None
            continue
        
        # For each available action in the current state
        for action in actions:
            # Calculate value of taking action in state
            p = {state: action}
            value = val(t, [p] + ps_tail, state)
            # Choose the action with the highest expected value
            if value > best_value:
                best_value = value
                best_action = action
        
        policy[state] = best_action

    return policy

In [26]:
# Builds an optimal policy sequence by recursively adding the best extension (starting from the end).
def bi(t: int, n: int) -> list[dict[str, str]]:
    if n == 0:
        return []
    else:
        ps_tail = bi(t + 1, n - 1)
        p = best_ext(t, ps_tail)
        return [p] + ps_tail

In [37]:
# For a given time step, state and decision horizon, returns the optimal action and the 
# expected value of the sequence it starts (assuming the rest of the sequence is optimal).
def best(t: int, n: int, x: str) -> str:
    if n == 0:
        raise(ValueError("The horizon must be greater than zero!"))
    ps = bi(t+1, n-1)
    p = best_ext(t, ps)
    b = p[x]
    vb = val(t, [p] + ps, x)
    return f"Horizon, best, value: {n}, {b}, {vb}"

In [38]:
# Computing the best decision for different decision horizons.
bests = []
for i in range(1,8):
    bests.append(best(0, i, "DHU"))

for best in bests:
    print(best)

Horizon, best, value: 1, Delay, 0.46799999999999997
Horizon, best, value: 2, Delay, 0.6354539999999999
Horizon, best, value: 3, Start, 0.9406696120000004
Horizon, best, value: 4, Start, 1.2500123183440004
Horizon, best, value: 5, Start, 1.5336353935581286
Horizon, best, value: 6, Start, 1.7907738537441182
Horizon, best, value: 7, Start, 2.0228744498053137


In [29]:
# Returns a value between 0 and 1, where 0 means "does not matter at all"
# and 1 means "matters maximally" to achieving the defined goal of the SDP.
def mMeas(t: int, n: int, x: str) -> float:
    if x in ["SHU", "SHC", "SLU", "SLC"]:
        return 0
    else:
        ps = bi(t, n)
        ps_prim = copy.deepcopy(ps)
        if ps[0][x] == "Start":
            ps_prim[0][x] = "Delay"
        else:
            ps_prim[0][x] = "Start"

        best_action_val = val(t, ps, x)
        worst_action_val = val(t, ps_prim, x)

        return (best_action_val - worst_action_val) / best_action_val

In [30]:
# Comparing mMeas values to those of the article
print(mMeas(0, 4, "SHU"))
print(mMeas(0, 6, "SLC"))
print(mMeas(0, 7, "DHU"))
print(mMeas(1, 7, "DHU"))
print(mMeas(3, 7, "DHU"))

0
0
0.17306026841327227
0.5673067719100584
0.5673067719100584
